In [1]:
import pandas as pd
from googletrans import Translator
import re
import numpy as np
import difflib

## Lista de EEUU

In [2]:
df_eeuu = pd.read_csv('datos_entrada_script_02/df_eeuu.csv').iloc[:,1:]
df_eeuu

,LastName,sdnType,Address,City,Country
0,AEROCARIBBEAN AIRLINES,enterprise,NaN,Havana,Cuba
1,"ANGLO-CARIBBEAN CO., LTD.",enterprise,"Ibex House, The Minories",London EC3N 1DY,United Kingdom
2,BANCO NACIONAL DE CUBA,enterprise,Zweierstrasse 35,Zurich CH-8022,Switzerland
3,BANCO NACIONAL DE CUBA,enterprise,Avenida de Concha Espina 8,Madrid E-28036,Spain
4,BANCO NACIONAL DE CUBA,enterprise,"Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...",Tokyo 103,Japan
...,...,...,...,...,...
15281,PZNFR TRADING LIMITED,enterprise,"Retail Lot L3-1, Level 3, Gateway Klia 2, Kual...","Sepang, Selangor Darul Ehsan 64000",Malaysia
15282,PIONEER SHIPMANAGEMENT PTE. LTD.,enterprise,"24-07, Shenton House, 3, Shenton Way",Singapore 068805,Singapore
15283,GOLDEN WARRIOR SHIPPING CO. LIMITED,enterprise,"9th Floor, Block C, Qingdao Plaza, 381, Dunhua...","Qingdao, Shandong 266034",China
15284,GOLDEN WARRIOR SHIPPING CO. LIMITED,enterprise,"Unit D, 16/F, One Capital Place, 18 Luard Road...",Hong Kong,China


Una vez identificadas las variables de interés de la lista de EEUU, se crea el DataFrame df_eeuu_t1 con estas variables. El 't1' en el nombre implica que esta es la primera transformación hecha sobre los datos tras ser cargados. Este DataFrame:

- Presenta una nueva variable llamada 'Origin' de valor 'EEUU' para identificar cual es la lista de origen.


- Se le han eliminado los registros con 'LastName' nulo. Ya que no podemos identificar un sancionado si no tiene un nombre.


- En caso de existir, se eliminan los registros duplicados. Para que se eliminen los registros duplicados pero que se diferencian por las mayúculas y minúsculas, se ponen todas la variables de tipo texto en mayúsculas.


- Se cambia el nombre de algunas columnas para darles un nombre más conciso. Este será el mismo nombre que se de a las variables de la lista de la UE para que puedan concatenarse sin problemas.

In [3]:
df_eeuu_t1 = df_eeuu.rename(columns={'sdnType':'EntityType','LastName':'Name'})

df_eeuu_t1.dropna(subset=['Name'],inplace=True)

df_eeuu_t1['Origin'] = 'EEUU'

for element in df_eeuu_t1.columns:
    df_eeuu_t1[element] = df_eeuu_t1[element].str.upper()
    
df_eeuu_t1.drop_duplicates(inplace=True)

df_eeuu_t1

,Name,EntityType,Address,City,Country,Origin
0,AEROCARIBBEAN AIRLINES,ENTERPRISE,NaN,HAVANA,CUBA,EEUU
1,"ANGLO-CARIBBEAN CO., LTD.",ENTERPRISE,"IBEX HOUSE, THE MINORIES",LONDON EC3N 1DY,UNITED KINGDOM,EEUU
2,BANCO NACIONAL DE CUBA,ENTERPRISE,ZWEIERSTRASSE 35,ZURICH CH-8022,SWITZERLAND,EEUU
3,BANCO NACIONAL DE CUBA,ENTERPRISE,AVENIDA DE CONCHA ESPINA 8,MADRID E-28036,SPAIN,EEUU
4,BANCO NACIONAL DE CUBA,ENTERPRISE,"DAI-ICHI BLDG. 6TH FLOOR, 10-2 NIHOMBASHI, 2-C...",TOKYO 103,JAPAN,EEUU
...,...,...,...,...,...,...
15281,PZNFR TRADING LIMITED,ENTERPRISE,"RETAIL LOT L3-1, LEVEL 3, GATEWAY KLIA 2, KUAL...","SEPANG, SELANGOR DARUL EHSAN 64000",MALAYSIA,EEUU
15282,PIONEER SHIPMANAGEMENT PTE. LTD.,ENTERPRISE,"24-07, SHENTON HOUSE, 3, SHENTON WAY",SINGAPORE 068805,SINGAPORE,EEUU
15283,GOLDEN WARRIOR SHIPPING CO. LIMITED,ENTERPRISE,"9TH FLOOR, BLOCK C, QINGDAO PLAZA, 381, DUNHUA...","QINGDAO, SHANDONG 266034",CHINA,EEUU
15284,GOLDEN WARRIOR SHIPPING CO. LIMITED,ENTERPRISE,"UNIT D, 16/F, ONE CAPITAL PLACE, 18 LUARD ROAD...",HONG KONG,CHINA,EEUU


En algunos valores de la columna 'City' aparecen códigos y números que no interesan. Se aplica una transformación mediante una expresión regular para eliminar estos códigos. Los valores transformados se almacenan en df_eeuu_t2. Ha sido necesario cambiar los valores nulos en 'City' a cadena de caracteres para poder escanearlos por la expresión regular. Una vez aplicada la expresión, se han vuelto a poner como valor NaN. Las pruebas para crear la expresión regular que eliminase las cadenas que no son de interés se han hecho en la página: https://regex101.com/r/OLJCjI/1

In [4]:
list_new_city=[]


for element in df_eeuu_t1.index:
    df = df_eeuu_t1.fillna({'City':'nulo'})
    string = df.loc[element,'City']
    new_string = re.sub(r"(, .{0,50})|( [A-Z]{0,16}-[0-9]{0,50})|( \w\w\d.{0,50}[0-9]{0,50})","", string)
    list_new_city.append(new_string)

In [5]:
df_eeuu_t2 = df_eeuu_t1.drop(columns='City')
df_eeuu_t2['City'] = list_new_city
df_eeuu_t2.replace('nulo',np.nan,inplace=True)
df_eeuu_t2

,Name,EntityType,Address,Country,Origin,City
0,AEROCARIBBEAN AIRLINES,ENTERPRISE,NaN,CUBA,EEUU,HAVANA
1,"ANGLO-CARIBBEAN CO., LTD.",ENTERPRISE,"IBEX HOUSE, THE MINORIES",UNITED KINGDOM,EEUU,LONDON
2,BANCO NACIONAL DE CUBA,ENTERPRISE,ZWEIERSTRASSE 35,SWITZERLAND,EEUU,ZURICH
3,BANCO NACIONAL DE CUBA,ENTERPRISE,AVENIDA DE CONCHA ESPINA 8,SPAIN,EEUU,MADRID
4,BANCO NACIONAL DE CUBA,ENTERPRISE,"DAI-ICHI BLDG. 6TH FLOOR, 10-2 NIHOMBASHI, 2-C...",JAPAN,EEUU,TOKYO
...,...,...,...,...,...,...
15281,PZNFR TRADING LIMITED,ENTERPRISE,"RETAIL LOT L3-1, LEVEL 3, GATEWAY KLIA 2, KUAL...",MALAYSIA,EEUU,SEPANG
15282,PIONEER SHIPMANAGEMENT PTE. LTD.,ENTERPRISE,"24-07, SHENTON HOUSE, 3, SHENTON WAY",SINGAPORE,EEUU,SINGAPORE
15283,GOLDEN WARRIOR SHIPPING CO. LIMITED,ENTERPRISE,"9TH FLOOR, BLOCK C, QINGDAO PLAZA, 381, DUNHUA...",CHINA,EEUU,QINGDAO
15284,GOLDEN WARRIOR SHIPPING CO. LIMITED,ENTERPRISE,"UNIT D, 16/F, ONE CAPITAL PLACE, 18 LUARD ROAD...",CHINA,EEUU,HONG KONG


## Lista de la UE

In [6]:
df_ue = pd.read_csv('datos_entrada_script_02/df_ue.csv').iloc[:,1:]
df_ue

,Entity_SubjectType_ClassificationCode,NameAlias_WholeName,Address_City,Address_CountryDescription,Address_Street
0,person,Saddam Hussein Al-Tikriti,NaN,NaN,NaN
1,person,Abu Ali,NaN,NaN,NaN
2,person,Abou Ali,NaN,NaN,NaN
3,person,NaN,NaN,NaN,NaN
4,person,NaN,NaN,NaN,NaN
...,...,...,...,...,...
21570,person,Viktor Fedorovych YANUKOVYCH,NaN,NaN,NaN
21571,person,Viktor Fedorovič JANUKOVIČ,NaN,NaN,NaN
21572,person,Viktor Fedorovici IANUKOVICI,NaN,NaN,NaN
21573,person,NaN,NaN,NaN,NaN


Una vez identificadas las variables de interés de la lista de la UE, se crea el DataFrame df_ue_t1 con estas variables. El 't1' en el nombre implica que esta es la primera transformación hecha sobre los datos tras ser cargados. Este DataFrame:

- Presenta una nueva variable llamada 'Origin' de valor 'UE' para identificar cual es la lista de origen.


- Se le han eliminado los registros con 'NameAlias_WholeName' nulo. Ya que no podemos identificar un sancionado si no tiene un nombre.


- Se eliminan los registros duplicados. Para que se eliminen los registros duplicados pero que se diferencian por las mayúculas y minúsculas, se ponen todas la variables de tipo texto en mayúsculas.


- Se cambia el nombre de algunas columnas para darles un nombre más conciso. Este será el mismo nombre que se ha dado a las variables de la lista de EEUU para que puedan concatenarse sin problemas.

In [7]:
df_ue_t1 = df_ue.rename(columns={'Entity_SubjectType_ClassificationCode':'EntityType',
                'NameAlias_WholeName':'Name','Address_City':'City',
                'Address_CountryDescription':'Country','Address_Street':'Address'})

df_ue_t1.dropna(subset=['Name'],inplace=True)

df_ue_t1['Origin'] = 'UE'

for element in df_ue_t1.columns:
    df_ue_t1[element] = df_ue_t1[element].str.upper()
    
df_ue_t1.drop_duplicates(inplace=True)

df_ue_t1 = df_ue_t1.reset_index(drop=True)
df_ue_t1

,EntityType,Name,City,Country,Address,Origin
0,PERSON,SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,NaN,UE
1,PERSON,ABU ALI,NaN,NaN,NaN,UE
2,PERSON,ABOU ALI,NaN,NaN,NaN,UE
3,PERSON,QUSAY SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,NaN,UE
4,PERSON,QOUSSAÏ SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,NaN,UE
...,...,...,...,...,...,...
14486,PERSON,OLEKSANDR VIKTOROVICI IANUKOVICI,NaN,NaN,NaN,UE
14487,PERSON,VIKTOR FEDOROVYTJ JANUKOVYTJ,NaN,NaN,NaN,UE
14488,PERSON,ВIКТОР ФЕДОРОВИЧ ЯНУКОВИЧ,NaN,NaN,NaN,UE
14489,PERSON,VIKTOR FEDOROVIČ JANUKOVIČ,NaN,NaN,NaN,UE


Se analizan todos los registros de la columna 'Name' mediante un bucle for para detectar posibles problemas en los valores:

In [9]:
for i in range(0,len(df_ue_t1)-1):
    print(df_ue_t1.loc[i,'Name'])

SADDAM HUSSEIN AL-TIKRITI
ABU ALI
ABOU ALI
QUSAY SADDAM HUSSEIN AL-TIKRITI
QOUSSAÏ SADDAM HUSSEIN AL-TIKRITI
UDAY SADDAM HUSSEIN AL-TIKRITI
OUDAÏ SADDAM HUSSEIN AL-TIKRITI
ABID HAMID MAHMUD AL-TIKRITI
ABID HAMID BID HAMID MAHMUD
ABDEL HAMID MAHMOUD
ABED MAHMOUD HAMMUD
ABID HAMID MAHMOUD AL-TIKRITI
ABID HAMID BID HAMID MAHMOUD
ABED MAHMOUD HAMMOUD
ALI HASSAN AL-MAJID AL-TIKRITI
AL-KIMAWI
IZZAT IBRAHIM AL-DURI
ABU BRAYS
ABU AHMAD
IZZAT IBRAHIM AL-DOURI
HANI ABD-AL-LATIF TILFAH AL-TIKRITI
HANI ABDEL-LATIF TILFAH AL-TIKRITI
MUHAMMAD HAMZA ZUBAIDI
MOHAMMED HAMZA ZOUBAÏDI
BARZAN ABD AL-GHAFUR SULAIMAN MAJID AL-TIKRITI
BARZAN RAZUKI ABD AL-GHAFUR
BARZAN ABDEL GHAFOUR SOULEIMAN MAJID AL-TIKRITI
BARZAN RAZUKI ABDEL GHAFOUR
MUZAHIM SA'B HASSAN AL-TIKRITI
MUZAHEM SAAB HASSAN AL-TIKRITI
IBRAHIM AHMAD ABD AL-SATTAR MUHAMMED AL-TIKRITI
IBRAHIM AHMED ABDEL SATTAR MOHAMMED AL-TIKRITI
SAIF-AL-DIN FULAYYIH HASSAN TAHA AL-RAWI
AYAD FUTAYYIH AL-RAWI
SAÏF-AL-DIN FOULAÏ HASSAN TAHA AL-RAOUI
AYAD FOUTAÏ AL-R

PARTIDO ISLÂMICO DO TURQUESTÃO
SAID BAHAJI
ZOUHEIR AL MAGHRIBI
MOHAMED ABBATTAY
ABDERRAHMANE AL MAGHRIBI
RAMZI MOHAMED ABDULLAH BINALSHIBH
RAMZI MOHAMED ABDELLAH OMAR
RAMZI MOHAMED ABDULLAH BINALSHEIDAH
RAMZI BIN AL SHIBH
MOHAMED ALI ABDULLAH BAWAZIR
RAMZI OMAR
RAMZI MOHAMMED ABDULLAH BINALSHIBH
RAMZI BINALSHIB
RAMZI MOHAMED ABDELLAH OMAR HASSAN ALASSIRI
RAMSI MOHAMED ABDULLAH BINALSHIBH
ABU UBAYDAH
"UMAR MUHAMMAD ""ABDALLAH BA"" AMAR"
MOUNIR EL MOTASSADEQ
MOUNIR EL MOUTASSADEQ
ZAKARYA ESSABAR
ZAKARIYA ESSABAR
MOROCCAN ISLAMIC COMBATANT GROUP
GICM
GRUPO ISLÁMICO COMBATIENTE MARROQUÍ
GRUPPO ISLAMICO COMBATTENTE MAROCCHINO
GROUPE ISLAMIQUE COMBATTANT MAROCAIN
MAROKKAANSE GROEP VAN ISLAMITISCHE STRIJDERS
GRUPO ISLÂMICO COMBATENTE MARROQUINO
TUNISIAN COMBATANT GROUP
GRUPO COMBATIENTE TUNECINO
GROUPE COMBATTANT TUNISIEN
GROUPE ISLAMISTE COMBATTANT TUNISIEN
GICT
TUNÉZIAI HARCOLÓ CSOPORTOK
TUNEZYJSKA GRUPA BOJOWA
ТУНИЗИЙСКА БОЙНА ГРУПА
GLOBAL RELIEF FOUNDATION (GRF)
JEMAAH ISLAMIYA
JEMA'AH IS

FORCES AÉRIENNES DU CORPS DES GARDIENS DE LA RÉVOLUTION ISLAMIQUE
FORZA AEREA DELL'IRGC
IRGC GAISA SPĒKI
ISLAMO REVOLIUCIJOS GVARDIJOS ORO PAJĖGOS
FORZA TA' L-AJRU IRGC
IRGC-LUCHTMACHT
SIŁY LOTNICZE IRGC
FORÇA AÉREA DO IRGC
VZDUŠNÉ SILY IRGC
ZRAČNE SILE IRGC
IRGC:N ILMAVOIMAT
KHATEM-OL ANBIYA CONSTRUCTION ORGANISATION
СТРОИТЕЛНА ОРГАНИЗАЦИЯ KHATEM-OL ANBIYA
STAVEBNÍ ORGANIZACE KHATEM-OL ANBIYA
KHATEM-OL ANBIYA STATYBOS ORGANIZACIJA
ORGANIZZAZZJONI TA' KOSTRUZZJONI KHATEM-OL ANBIYA
KHATEM-OL ANBIYA ORGANISATIE VAN DE BOUWSECTOR
STAVEBNÁ ORGANIZÁCIA KHATEM-OL ANBIYA
KHATAM AL-ANBIYA CONSTRUCTION HEADQUARTERS
KAA
MALEK ASHTAR UNIVERSITY
УНИВЕРСИТЕТ MALEK ASHTAR
UNIVERZITA MALEKA AŠTARA
MALEK ASHTAR-UNIVERSITÄT
ΠΑΝΕΠΙΣΤΉΜΙΟ MALEK ASHTAR
UNIVERSITÉ MALEK ASHTAR
MALEK ASHTAR UNIVERSITETAS
UNIVERSITÀ MALEK ASHTAR
UNIVERSIDADE MALEK ASHTAR
UNIVERSITATEA MALEK ASHTAR
UNIVERZITA MALEK ASHTAR
UNIVERZA ZA OBRAMBNO TEHNOLOGIJO MALEK ASHTAR
MECHANIC INDUSTRIES GROUP
SKUPINA STROJNÍHO PRŮMYSLU
GRUPP 

BEHROOZ_ICE
MOUSSA KHALILOLLAHI
MOUSA KHALILOLLAHI
MOUSA KHALIL ELAHI
SADEQ MAHSOULI
SADEQ MAHSULI
MOJTABA MALEKI
MEHRDAD OMIDI
REZA
REZA OMIDI
MAHMOUD SALARKIA
HOJATOLLAH KHODAEI SOURI
HOSSEIN TALA
HOSSEYN TALA
MORTEZA TAMADDON
MORTEZA TAMADON
HOSSEIN ZEBHI
MOHAMMAD-KAZEM BAHRAMI
AZIZ HAJMOHAM-MADI
NOOROLLAH AZIZMOHAMMADI
AZIZ HAJMOHAMMADI
MOHAMMAD-BAGHER BAGHERI
SEYYED MORTEZA BAKHTIARI
MOHAMMAD HOSSEINI
SEYYED MOHAMMAD HOSSEYNI
SEYED MOHAMMAD HOSSEYNI
SAYYED MOHAMMAD HOSSEYNI
SAYYID MOHAMMAD HOSSEYNI
HEYDAR MOSLEHI
HEIDAR MOSLEHI
HAIDAR MOSLEHI
COMMERCIAL BANK OF SYRIA
BANCO COMERCIAL DE SIRIA
ТЪРГОВСКА БАНКА НА СИРИЯ
BANCA COMERCIALĂ A SIRIEI
SZÍRIAI KERESKEDELMI BANK
OBCHODNÁ BANKA SÝRIE
SYRSKÁ OBCHODNÍ BANKA
SIRIJOS KOMERCIJOS BANKAS
SĪRIJAS KOMERCBANKA
SÜÜRIA KOMMERTSPANK
AL-MASRAF AL- TAJARIYY AL- SURIYY
ABDOLLAHI HAMED
MUSTAFA ABDULLAHI
ARBABSIAR MANSSOR
MANSOUR ARBABSIAR
SHAHLAI ABDUL REZA
ABDOL REZA SHALA'I
ABD-AL REZA SHALAI
ABDORREZA SHAHLAI
ABDOLREZA SHAHLA'I
ABDUL-REZA S

RAMA MILITAR DE HIZBU'LLAH
RAMA MILITAR DE HIZB ALLAH
CONSEJO DE LA JIHAD
ALA MILITAR DO HEZBOLÁ
CONSELHO DA JIHAD
ZBROJNE RAMIĘ HEZBOLLAHU
ALA MILITARE DI HEZBOLLAH
CONSIGLIO DELLA JIHAD
HIZBOLLAHS MILITÆRE GREN
ARIPA MILITARĂ A HEZBOLLAHULUI
CONSILIUL JIHADULUI
MILITAIRE VLEUGEL VAN HIZB ALLAH
MILITAIRE VLEUGEL VAN HIZBU'LLAH
MILITAIRE VLEUGEL VAN HISBOLLAH
MILITAIRE VLEUGEL VAN HEZBALLAH
MILITAIRE VLEUGEL VAN HIZBOLLAH
MILITAIRE VLEUGEL VAN HIZBULLAH
MILITAIRE VLEUGEL VAN HIZBALLAH
EXTERNAL SECURITY ORGANISATION
HIZBOLLAHS VÄPNADE GREN
VOJAŠKO KRILO HEZBOLAHA
VOJENSKÉ KRÍDLO HIZBALLÁHU
CONSELHO DA JIADE
RADA DŻIHADU
WOJSKOWE SKRZYDŁO HIZBALLAH
"L-""KUNSILL TAL-ĠIHAD"
IL-FERGĦA MILITARI TA' HIZBALLAH
DZSIHÁD TANÁCS
HIZBALLAH KATONAI SZÁRNY
DŽIHADO TARYBA
HIZBALLAH KARINIS SPARNAS
VIJEĆE DŽIHADA
VOJNO KRILO HEZBOLAHA
COMHAIRLE JIOHÁD
SCIATHÁN MÍLEATA HIZBOLLAH
BRANCHE MILITAIRE DU HEZBOLLAH
ΣΥΜΒΟΎΛΙΟ ΤΗΣ ΤΖΙΧΆΝΤ
ΣΤΡΑΤΙΩΤΙΚΉ ΠΤΈΡΥΓΑ ΤΗΣ ΧΕΖΜΠΟΛΆΧ
DŽIHAADI NÕUKOGU
HIZBALLAHI SÕJALINE TI

...

VADIM SERGEYEVICH PRIGARA
VADIM SERGEJEVITJ PRIGARA
VADZIM SIARHEJEVITJ PRYHARA
АЛЕКСАНДР АЛЕКСАНДРОВИЧ ПЕТРАШ
АЛЯКСАНДР АЛЯКСАНДРАВІЧ ПЕТРАШ
ALEXANDR ALEXANDROVICH PETRASH
ALIAKSANDR ALIAKSANDRAVICH PIETRASH
ALEKSANDR ALEKSANDROVITJ PETRASJ
ALJAKSANDR ALJAKSANDRAVITJ PETRASJ
ALEXANDER ALEXANDROVICH PETRASH
АНДРЕЙ АЛЕКСАНДРОВИЧ ЛАГУНОВИЧ
АНДРЭЙ АЛЯКСАНДРАВІЧ ЛАГУНОВІЧ
ANDREI ALEXANDROVICH LAHUNOVICH
ANDREI ALIAKSANDRAVICH LAHUNOVICH
ANDREJ ALEKSANDROVITJ LAGUNOVITJ
ANDREJ ALJAKSANDRAVITJ LAHUNOVITJ
ЕЛЕНА ВАСИЛЬЕВНА ЛИТВИНА
АЛЕНА ВАСІЛЬЕЎНА ЛІТВІНА
ELENA VASILEVNA LITVINA
ALENA VASILEUNA LITVINA
JELENA VASILJEVNA LITVINA
ALENA VASILJEUNA LITVINA
ВИКТОРИЯ ВАЛЕРЬЕВНА ШАБУНЯ
ВІКТОРЫЯ ВАЛЕР'ЕЎНА ШАБУНЯ
VICTORIA VALEREVNA SHABUNYA
VICTORIA VALERYEUNA SHABUNYA
VIKTORIJA VALERJEVNA SJABUNJA
VIKTORYJA VALERJEUNA SJABUNJA
ЕЛЕНА АЛЕКСАНДРОВНА ЖИВИЦА
АЛЕНА АЛЯКСАНДРАВНА ЖЫВІЦА
ELENA ALEXANDROVNA ZHYVITSA
ALENA ALIAKSANDRAVNA ZHYVITSA
JELENA ALEKSANDROVNA ZJIVITSA
ALENA ALJAKSANDRAVNA ZJYVITSA
НАТА

ATVIROJI AKCINĖ BENDROVĖ „BELSHINA“
ABU JIHAD TNT
EMRAAN ALI
АНДРЕЙ НИКОЛАЕВИЧ ТРОШЕВ
ANDREI  MYKOLAYVYCH  TROSHEV
ANDREY NIKOLAEVICH  TROSHEV
ANDREJ NIKOLAJEVITJ TROSJEV
АНДРЕЙ МИХАЙЛОВИЧ БОГАТОВ
ANDREI  MYCHAILOVYCH  BOGATOV
ANDREY MIKHAILOVICH  BOGATOV
ANDREJ MICHAJLOVITJ BOGATOV
VELADA LLC
ООО ВЕЛАДА
MERCURY LLC
OOO МЕРКУРИЙ
EVRO POLIS LLC
ООО ЕВРО ПОЛИС
ДМИТРИЙ ВАЛЕРЬЕВИЧ  УТКИН
DIMITRIY VALERIEVICH UTKIN
DMITRY VALERIEVICH UTKIN
DMITRI VALERIEVICH UTKIN
DIMITRY VALERIEVICH UTKIN
DMITRIJ VALERJEVITJ  UTKIN
ДЕНИС ЮРЬЕВИЧ  ХАРИТОНОВ
DENIS YURIEVICH KHARITONOV
DENIS  JURJEVITJ KHARITONOV
СЕРГЕЙ ВЛАДИМИРОВИЧ  ЩЕРБАКОВ
SERGEY  VLADIMIROVICH  SHCHERBAKOV
SERGEJ  VLADIMIROVITJ  SJTJERBAKOV
АЛЕКСАНДР СЕРГЕЕВИЧ КУЗНЕЦОВ
ALEXANDER SERGEEVICH KUZNETSOV
ALEKSANDR SERGEEVICH KUZNETSOV
ALEKSANDR  SERGEJEVITJ KUZNETSOV
ДМИТРИЙ ВАЛЕРЬЕВИЧ УТКИН
DMITRY VALERIEVICH  UTKIN
DMITRI VALERIEVICH  UTKIN
DIMITRY VALERIEVICH  UTKIN
DIMITRIY VALERIEVICH  UTKIN
DMITRIJ VALERJEVITJ UTKIN
СТАНИСЛАВ ЕВГЕНЬЕВИЧ 

ПАВЕЛ ВЛАДИМИРОВИЧ  КРАШЕНИННИКОВ
PAVEL VLADIMIROVICH KRASHENINNIKOV
PAVEL VLADIMIROVITJ KRASJENINNIKOV
АННА  ЮРЬЕВНА КУЗНЕЦОВА
ANNA YURYEVNA KUZNETSOVA
ANNA JURJEVNA KUZNETSOVA
РИЗВАН  ДАНИЯЛОВИЧ КУРБАНОВ
RIZVAN DANILOVICH KURBANOV
RIZVAN DANILOVITJ KURBANOV
ОЛЕГ ЮРЬЕВИЧ ЛЕОНОВ
OLEG YURYEVICH LEONOV
OLEG JURJEVITJ LEONOV
АНДРЕЙ  МИХАЙЛОВИЧ МАКАРОВ
ANDREY MIKHAILOVICH MAKAROV
ANDREJ MIKHAJLOVITJ MAKAROV
ЕВГЕНИЙ  ЕВГЕНЬЕВИЧ МАРЧЕНКО
EVGENY EVGENIEVICH MARCHENKO
JEVGENIJ JEVGENJEVITJ MARTJENKO
ОЛЕГ  АНАТОЛЬЕВИЧ МАТВЕЙЧЕВ
OLEG ANATOLYEVICH MATVEYCHEV
OLEG ANATOLJEVITJ MATVEJTJEV
АРТЁМ  ПАВЛОВИЧ МЕТЕЛЕВ
ARTEM PAVLOVICH METELEV
ARTIOM PAVLOVITJ METELEV
ОЛЕГ  ВИКТОРОВИЧ МОРОЗОВ
OLEG VIKTOROVICH  MOROZOV
OLEG VIKTOROVITJ MOROZOV
ЕВГЕНИЙ  CЕРГЕЕВИЧ МОСКВИЧЕВ
YEVGENY SERGEYEVICH MOSKVICHEV
JEVGENIJ SERGEJEVITJ MOSKVITJOV
АЛЕКСЕЙ  ГЕННАДЬЕВИЧ НЕЧАЕВ
ALEXEY GENNADYEVICH NECHAYEV
ALEKSEJ GENNADJEVITJ NETJAJEV
ВИКТОРИЯ  ВИКТОРОВНА НИКОЛАЕВА
VIKTORIA VIKTOROVNA NIKOLAEVA
VIKTORIA VIKTOROVNA NIKOLAJE

ANDREY SERGEEVICH VOROSHILOV
ANDRIJ SERHIJOVYTJ VOROSJYLOV
ANDREJ SERGEJEVITJ VOROSJILOV
ОЛЕКСАНДР ПАВЛОВИЧ КУРЄНКОВ
АЛЕКСАНДР ПАВЛОВИЧ КУРЕНКОВ
ALEXANDER PAVLOVICH KURENKOV
OLEKSANDR PAVLOVYTJ KURJENKOV
ALEKSANDR PAVLOVITJ KURENKOV
ВЛАДІСЛАВ ЛЕОНІДОВИЧ БЕРДIЧЕВСЬКИЙ
ВЛАДИСЛАВ ЛЕОНИДОВИЧ БЕРДИЧЕВСКИЙ
VLADISLAV LEONIDOVICH BERDICHEVSKY
VLADISLAV LEONIDOVYTJ BERDITJEVSKYJ
VLADISLAV LEONIDOVITJ BERDITJEVSKIJ
ІРИНА ВАСИЛІВНА ПОПОВА
ИРИНА ВАСИЛЬЕВНА ПОПОВА
IRINA VASILIEVNA POPOVA
IRYNA VASYLIVNA POPOVA
IRINA VASILJEVNA POPOVA
МАРІЯ ВОЛОДИМИРІВНА ПIРОГОВА
МАРИЯ ВЛАДИМИРОВНА ПИРОГОВА
MARIA VLADIMIROVNA PIROGOVA
MARIJA VOLODYMYRIVNA PIROHOVA
MARIJA VLADIMIROVA PIROGOVA
СЕРГІЙ БОРИСОВИЧ ПРОКОПЕНКО
СЕРГЕЙ БОРИСОВИЧ ПРОКОПЕНКО
SERGEY BORISOVICH PROKOPENKO
SERHIJ BORYSOVYTJ PROKOPENKO
SERGEJ BORISOVITJ PROKOPENKO
СВІТЛАНА АНАТОЛІЇВНА КУМАНОВА
СВЕТЛАНА АНАТОЛЬЕВНА КУМАНОВА
SVETLANA ANATOLIEVNA KUMANOVA
SVITLANA ANATOLIJIVNA KUMANOVA
SVETLANA ANATOLJEVNA KUMANOVA
МАГДАЛИНА МАРИНА ВОЛОДИМИРIВНА
МАГДА

ASSOCIAÇÃO  KVANT DE INVESTIGAÇÃO E  PRODUÇÃO
VENNOOTSCHAP OP AANDELEN  ONDERZOEKS- EN  PRODUCTIEVERENIGING KVANT
JSC FORSCHUNGS- UND PRODUKTIONSVEREINIGUNG  KVANT
АКЦИОНЕРНО ДРУЖЕСТВО  НАУЧНО-ПРОИЗВОДСТВЕНА  АСОЦИАЦИЯ КВАНТ
ЮНАРМИЯ
YUNARMIYA
ВСЕРОССИЙСКОЕ ВОЕННО ПАТРИОТИЧЕСКОЕ  ОБЩЕСТВЕННОЕ ДВИЖЕНИЕ  ЮНАРМИЯ
ALL-RUSSIA YOUNG ARMY MILITARY PATRIOTIC SOCIAL  MOVEMENT
ASOCIAȚIA PANRUSĂ  PRIVIND MIȘCAREA PATRIOTICĂ  MILITARO-SOCIALĂ DE LA NIVEL  NAȚIONAL DIN ÎNTREAGA RUSIE  TÂNĂRA ARMATĂ
ASSOCIAÇÃO PAN RUSSA DO MOVIMENTO SOCIAL  MILITAR-PATRIÓTICO JOVEM  EXÉRCITO
PAN-RUSSISCHE JONG LEGER MILITAIRE PATRIOTTISCHE EN SOCIALE BEWEGING
ÖSSZOROSZ IFJÚ HADSEREG KATONAI HAZAFIAS TÁRSADALMI MOZGALOM
VISOS RUSIJOS KARINIS-PATRIOTINIS  VISUOMENINIS JUDĖJIMAS  JAUNOJI ARMIJA
MOUVEMENT SOCIAL  PATRIOTIQUE MILITAIRE  PANRUSSE DE LA JEUNE  ARMÉE
JUNARMIJA
ALLRUSSISCHE MILITÄRISCH- PATRIOTISCHE GESELLSCHAFTLICHE BEWEGUNG JUGENDARMEE
MOVIMIENTO SOCIAL PATRIÓTICO  MILITAR PANRUSO DEL EJÉRCITO  JOVEN
ОБЩОРУ

Todavía hay dos problemas con la columna 'Name':

1. Existen nombres escritos de forma muy similar que podían ser duplicados.

2. Existen nombres iguales pero escritos en distintos idiomas que podrían ser duplicados.

Para el problema del punto 1 se crea una segunda transformación sobre el DataFrame y se renombra como df_ue_t2. Se emplea la función 'get_close_matches()' del módulo de Python llamado difflib. 

La función se emplea dentro de un bucle for que comprueba para cada registro cuantos registros similares o iguales tiene debajo. Si se detecta que no hay ningún registro similar, este es mandado a la lista 'new_list_name' con un condicional de tipo if. Cada elemento en new_list_name representa un registro con las columnas separadas por guiones. La función presenta dos parámetros ajustables:

- 'n' tiene valor 4 por defecto e indica el número máximo de registros que, definido un cierto nombre, pueden ser similares a este. Es decir, para cada nombre el número máximo de nombres que pueden considerarse duplicados debido a su similitud y por tanto pueden ser eliminados es de 4 nombres.


- 'sensibility' es un número entre 0 y 1 que indica el grado de sensibilidad con el que funciona el detector de matches. A mayor sensibilidad, más registros serán considerados como duplicados y, en consecuencia, menos registros aparecen en la lista final.

In [ ]:
df_ue_t1.fillna('0',inplace=True)

list_name = df_ue_t1['Name']+';'+df_ue_t1['City']+';'+df_ue_t1['Country']+';'+df_ue_t1['Address']+';'+df_ue_t1['EntityType']+';'+df_ue_t1['Origin']

new_list_name = []

n = 4
sensibility = 0.2

for i in list(range(len(list_name))):
    
    word = list_name[i]
    possibilities = list_name[(i+1):]
    
    close_matches = difflib.get_close_matches(word, possibilities, n, 1-sensibility)
    
    if len(close_matches) == 0:
        new_list_name.append(list_name[i])
new_list_name

In [ ]:
df_ue_t2 = pd.DataFrame(new_list_name)
df_ue_t2.rename(columns={0:'column'},inplace=True)
df_ue_t2 = df_ue_t2.column.str.split(pat=';',expand=True)
df_ue_t2 = df_ue_t2.rename(columns={0:'Name',1:'City',2:'Country',3:'Address',4:'EntityType',5:'Origin'})


Para el problema del punto 2 se crea una tercera transformación sobre el DataFrame y se renombra como df_ue_t3. Esta transformación traduce los nombres al inglés. De esta forma se podrán eliminar los registros duplicados escritos en distintos idiomas al aplicar la función drop_duplicates().

- T1_1: QUITAR EL CP DE CITY, eliminar registros muy parecidos pero con pequeñas diferencias en el nombre, traducir los registros para eliminar duplicados (t. para dashboard).

- T1_2: Imputaciones. Eliminar registros muy parecidos 

- T1_3: imputaciones con eliminado básico de duplicados (t. para machine learning supervisado)

Usar replace para pasar los nulos de origen a nulos de python